<a href="https://colab.research.google.com/github/sandeep92134/The-Data-Science-Workshop-By-Packt/blob/master/module%2015/Activity15.02%3A%20Comparison%20of%20Advanced%20Ensemble%20Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comparison of Advanced Ensemble Techniques
Scenario: You have tried the benchmark model on the credit card dataset and have got some benchmark metrics. Having learned some advanced ensemble techniques, you want to determine which technique to use for the credit card approval dataset.

In this activity, you will use all three advanced techniques and compare the results before selecting your final technique.

The steps are as follows:

1. Open a new Colab notebook.
2. Implement all steps from Exercise 15.01, Loading, Exploring, and Cleaning the Data, up until the splitting of the dataset into train and test sets.
3. Implement the bagging technique with the base learner as the logistic regression model. In the bagging classifier, define n_estimators = 15, max_samples = 0.7, and max_features = 0.8. Fit the model on the training set, generate the predictions, and print the confusion matrix and the classification report.
4. Implement boosting with random forest as the base learner. In the AdaBoostClassifier, define n_estimators = 300. Fit the model on the training set, generate the predictions, and print the confusion matrix and classification report.
5. Implement the stacking technique. Make the KNN and logistic regression models base learners and random forest a meta learner. Fit the model on the training set, generate the predictions, and print the confusion matrix and classification report.
6. Compare the results across all three techniques and select the best technique.
7. Output: You should get an output similar to the following for all three methods. Please note you will not get exact values as output due to variability in the prediction process.

In [1]:
import pandas as pd

In [2]:
#Loading data from the Github repository to colab notebook
filename = 'https://raw.githubusercontent.com/sandeep92134/The-Data-Science-Workshop-By-Packt/master/module%2015/datasets/crx.data'

In [3]:
# Loading the data using pandas
credData = pd.read_csv(filename,sep=",",header = None,na_values = "?")
credData.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [4]:
# Changing the Classes to 1 & 0
credData.loc[credData[15] == '+' , 15] = 1
credData.loc[credData[15] == '-' , 15] = 0
credData.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [5]:
# Dropping all the rows with na values
newcred = credData.dropna(axis = 0)
newcred.shape

(653, 16)

In [6]:
# Seperating the categorical variables to make dummy variables
credCat = pd.get_dummies(newcred[[0,3,4,5,6,8,9,11,12]])


In [7]:
# Seperating the numerical variables
credNum = newcred[[1,2,7,10,13,14]]


In [8]:
# Making the X variable which is a concatenation of categorical and numerical data
X = pd.concat([credCat,credNum],axis = 1)
print(X.shape)

# Seperating the label as y variable
y = pd.Series(newcred[15], dtype="int")
print(y.shape)

(653, 46)
(653,)


In [9]:
# Normalising the data sets
# Import library function
from sklearn import preprocessing
# Creating the scaling function
minmaxScaler = preprocessing.MinMaxScaler()
# Transforming with the scaler function
X_tran = pd.DataFrame(minmaxScaler.fit_transform(X))
# Printing the output
X_tran.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.271111,0.000000,0.043860,0.014925,0.1010,0.00000
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.713016,0.159286,0.106667,0.089552,0.0215,0.00560
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.170635,0.017857,0.052632,0.000000,0.1400,0.00824
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.223492,0.055000,0.131579,0.074627,0.0500,0.00003
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.101905,0.200893,0.060000,0.000000,0.0600,0.00000


In [10]:
# Splitting the data set to train and test sets
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tran, y, test_size=0.3, random_state=123)



**Bagging**

In [11]:
# Defining the base learner
from sklearn.linear_model import LogisticRegression

bl1 = LogisticRegression(random_state=123) 

In [12]:
# Creating the bagging meta learner

from sklearn.ensemble import BaggingClassifier

baggingLearner = BaggingClassifier(base_estimator=bl1, n_estimators=15, max_samples=0.7, max_features=0.8)

In [13]:
# Fitting the model using the meta learner
model = baggingLearner.fit(X_train, y_train)

In [14]:
# Predicting on the test set using the model
pred = model.predict(X_test)

In [15]:
# Printing the confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, pred))



[[92 15]
 [ 8 81]]


In [16]:
# Printing the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       107
           1       0.84      0.91      0.88        89

    accuracy                           0.88       196
   macro avg       0.88      0.88      0.88       196
weighted avg       0.89      0.88      0.88       196



**Boosting**

In [17]:
# Defining the base learner
from sklearn.ensemble import RandomForestClassifier

bl1 = RandomForestClassifier(random_state=123)

In [18]:
# Defining the boosting meta learner
from sklearn.ensemble import AdaBoostClassifier

boosting = AdaBoostClassifier(base_estimator=bl1, n_estimators=300)

In [19]:
# Fitting the model on the training set
model = boosting.fit(X_train, y_train)

In [20]:
# Getting the predictions from the boosting model
pred = model.predict(X_test)

In [21]:
# Printing the confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, pred))

# Printing the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

[[95 12]
 [ 9 80]]
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       107
           1       0.87      0.90      0.88        89

    accuracy                           0.89       196
   macro avg       0.89      0.89      0.89       196
weighted avg       0.89      0.89      0.89       196



**Stacking**

In [22]:
# Importing the meta learner and base learners
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

bl1 = KNeighborsClassifier(n_neighbors=5)
bl2 = LogisticRegression(random_state=123) 
ml = RandomForestClassifier(random_state=123)

In [23]:
# Creating the stacking classifier
from mlxtend.classifier import StackingClassifier
stackclf = StackingClassifier(classifiers=[bl1, bl2], 
                          meta_classifier=ml)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [24]:
# Fitting the model on the training set
model = stackclf.fit(X_train, y_train)


In [25]:
# Generating predictions on test set
pred = model.predict(X_test)

In [26]:
# Printing the confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred))

# Printing the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

[[99  8]
 [18 71]]
              precision    recall  f1-score   support

           0       0.85      0.93      0.88       107
           1       0.90      0.80      0.85        89

    accuracy                           0.87       196
   macro avg       0.87      0.86      0.86       196
weighted avg       0.87      0.87      0.87       196

